Delta Debugging
=========================

Delta Debugging is a methodology that automates program debugging using a scientific approach of hypothesis-trial-result loop. It was created in 1999 when there were many open bugs in Mozilla's bug database. Each bug describes a scenario which caused software to fail. The bug reports could contain a lot of irrelevant information and many of them could be equivalent. It would be very helpful in finding the error if we could simplify the input and still generate the same failures. So we follow the general pattern to put Delta Debugging to work:


1. **Identify the test case(s).**

2. **Identify the deltas.**

3. **Write a testing function.**

4. **Invoke Delta Debugging.**

## Hands On
<a href="https://colab.research.google.com/github/damorimRG/practical_testing_book/blob/master/debugging/deltadebugging.ipynb" target="_blank"> 
<img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"></a>

### **1 - Identify the test case(s)**

Our program consists of **4** small functions and a **prod function** that accepts a list of values and returns their product ( $\Pi$ ).

It fails because it returns a **NaN** and we want to investigate why.

In order to complete this step **we need to meet the following requirements**:

1. Define a test case where your program fails:
    - Calling our prod function with all the items on the list where we get a **NaN** return.


2. Define a test case where this failure does not happen:
    - Calling our prod function with one item in the list, where it returns a valid value.

In [1]:
from decimal import Decimal

def functionA(): 
    return 1

def functionB(): 
    return 2

def functionC(): 
    return 3

def functionD():
    return 4

def functionE():
    return Decimal('nan')

def prod(args):
    res = 1
    for a in args:
        res *= a
    return res

In [2]:
A = prod([functionA(), functionB(), functionC(), functionD(), functionE()])
B = prod([functionB()])
print(f'A: {A}\nB: {B}')

A: NaN
B: 2


### **2 - Identify the deltas.**

For the next step we must find a set of deltas, which is the difference between the entry in the failed execution and the entry in the execution without failure.

In our case the entries are:


1. `[functionA(), functionB(), functionC(), functionD(), functionE()]`
2. `[functionB()]`

So the difference is: `[functionA(), functionC(), functionD(), functionE()]`

Then we can divide our set of deltas into small parts later.

In [3]:
deltas = [functionA(), functionC(), functionD(), functionE()]

### **3 - Write a testing function.**

In this stage of the tests, we have a test function that receives a list of deltas:
1. Prints PASS if the result is an integer. 
1. Prints FAIL if the result is not an integer.

In [4]:
def test(deltas):
    for i in range(0, len(deltas)):
        result = prod(deltas[0:i+1])
        if isinstance(result, int):
            print("PASS")
        else:
            print("FAIL")

In [5]:
test(deltas)

PASS
PASS
PASS
FAIL


### **4 - Invoke Delta Debugging.**

Now that we have the test function, we want to identify which function is the problem. We will then run the delta debugging and it will show us the results produced for each input. This way we can identify wich function is breaking the program.

To make our job easier, let's make a small change to our test function.

We will create a python dictionary, and store the inputs and results of our **prod** function as a key-value pair.

In [6]:
def test(deltas):
    ans = dict()
    for i in range(0,len(deltas)):
        result = prod(deltas[0:i+1])
        if isinstance(result, int):
            ans[tuple(deltas[0:i+1])] = "PASS"
        else:
            ans[tuple(deltas[0:i+1])] = "FAIL"
    return ans

In [7]:
def deltaDebugging():
    ans = test(deltas)
    for key, value in ans.items():
        print(value, key)


So calling deltaDebuggig we have:


In [8]:
deltaDebugging()

PASS (1,)
PASS (1, 3)
PASS (1, 3, 4)
FAIL (1, 3, 4, Decimal('NaN'))


Where, as a result, we can see what our main program has entered and what it has given us. With this, looking at the last line, it is clear that when we pass the **functionE** to our **prod** function, a failure results. With that the biggest entry we can have is:

In [9]:
prod([functionA(), functionB(), functionC(), functionD()])

24

## **Conclusion**

We saw here how the delta debugging methodology works with a simple example to make the explanation clearer and more practical, as delta debugging is a methodology it can be implemented in several ways, there's a reference below with another implementation.

## **References**

*   [Using DeltaDebugging](https://www.st.cs.uni-saarland.de/dd/ddusage.php3)



